# Using the SAP IBP Connector

Practical guide on how to get started

## 1. Introduction

### What does this package contain
- Connector for connecting with SAP IBP
- Downloaders for downloading data from various IBP sources (currently: Keyfigure, masterdata & change history)
- Uploaders for uploading data to IBP (currently only Keyfigure)
- Query API for constructing queries compatible with the SAP IBP API. The Query & Downloader objects are integrated.

### Downloading data from SAP IBP
In order to download data from SAP IBP, use the various DataDownloader objects (e.g., KeyfigureDataDownloader and MasterdataDownloader). Downloaders return the requested data as a pandas dataframe, along with the response object from the requests library. View the specific downloader documentation to get information on what 'downloader-specific' information it requires.

#### Basic usage
```python
from sap_ibp_connector import IBPConnector
from sap_ibp_connector import (
    KeyfigureDataDownloader,
    MasterdataDownloader,
    ChangeHistoryDownloader,
)

# Establish connection using IBPConnector
with IBPConnector(username=username, password=password, planning_area=pa, url=url) as connector:
    # Instantiate data downloader
    downloader = MasterdataDownloader(connector, masterdata_table=my_table)

    df, response = downloader.download()

```

Downloading is also supported in chunks. This will return a concatenated dataframe of all downloaded chunks and a list of response objects per request.

### Uploading data to SAP IBP
The API supports uploading data to IBP. Data has to be send in json format, but uploaders provide support for converting pandas dataframes to a valid json to be send to IBP. You can provide the uploaders with a json and it will send it to IBP, if you provide a pandas Dataframe it will first be converted to json. Uploaders return a requests.Response object

#### Basic usage
```python
from sap_ibp_connector import IBPConnector
from sap_ibp_connector import KeyfigureDataUploader

# Establish connection using IBPConnector
with IBPConnector(username=username, password=password, planning_area=pa, url=url) as connector:
    # Instantiate data uploader
    uploader = KeyfigureDataUploader(connector)

    # uploaders expose functionality to convert pandas dataframe to IBP compatible json
    df_json = uploader.df_to_json(df)

    # upload the json
    response = uploader.upload(df_json)

    # if you provide a dataframe, DataUploader will call df_to_json automatically
    response = uploader.upload(df)
```

### Constructing queries
For downloading data, one can pass queries to DataDownloaders to select, filter & sort the requested data. Query strings as documented by the SAP API can directly be provided. This package also provides helpers for constructing queries.

Supported queries are:
- filter
- AND
- OR
- select
- top
- skip
- order_by

#### Basic usage

```python
from sap_ibp_connector.query import Query, select, filter
from sap_ibp_connector import KeyfigureDataDownloader, IBPConnector

# construct a query
query = Query(
    select("col1", "col2", "col3"),
    filter.ge("col1", 5)
)

# Query objects can directly be provided to DataDownloader objects
with IBPConnector(username=username, password=password, planning_area=pa, url=url) as connector:
    # Instantiate data downloader
    downloader = KeyfigureDataDownloader(connector)

    df, response = downloader.download(query=query)
```

You can combine queries by constructing a new Query object:

```python
query = Query(select_query, filter_query)

# with a custom query
query = Query(select_query, 'my_custom_query')
```

Or by using the "+" operator:

```python
query = select_query + filter_query

# with a custom query
query = select_query + 'my_custom_query'
```

Use AND/OR to combine filter queries:

```python
from sap_ibp_connector.query import filter, AND

query = AND(filter.eq("col1", "value"), filter.eq("col2", "other_value"))
```

Queries can be nested:

```python
from sap_ibp_connector.query import filter, AND, order_by, select

query = Query(
    select("col1", "col2", "col3"),
    Query(
            AND(filter.eq("col1", "value"), filter.eq("col2", "other_value")),
            order_by(col3='asc')
        )
    )
```

The query module also provides some helpers for constructing datetime queries:
- py_datetime_to_query_date
- string_to_query_date

### Link to SAP IBP documentation around OData integration services

https://help.sap.com/docs/SAP_INTEGRATED_BUSINESS_PLANNING/da797ae2bf6246d58abd417f24915d55/62f7075855409344e10000000a4450e5.html?locale=en-US

## Import required packages

In [2]:
# import dataiku
# from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np

# import core functionality of the sap ibp connector
from sap_ibp_connector import KeyfigureDataDownloader, MasterdataDownloader, KeyfigureDataUploader, IBPConnector

# import helpers to construct your query
from sap_ibp_connector.query import (
    Query,
    filter,
    select,
    skip,
    top,
    order_by,
    AND,
    OR,
    OrderDirections,
    string_to_query_date,
    py_datetime_to_query_date,
)# importing required packages
# import dataiku
# from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np

# import core functionality of the sap ibp connector
from sap_ibp_connector import KeyfigureDataDownloader, MasterdataDownloader, KeyfigureDataUploader, IBPConnector

# import helpers to construct your query
from sap_ibp_connector.query import (
    Query,
    filter,
    select,
    skip,
    top,
    order_by,
    AND,
    OR,
    OrderDirections,
    string_to_query_date,
    py_datetime_to_query_date,
)

## Example of reading key-figure data

In [3]:
# CREDENTIALS
IBP_USERNAME_MD_KF = ...
IBP_PASSWORD_MD_KF = ...
IBP_URL_KF = ...
IBP_PLANNING_AREA = "ZAPPLESI3"

In [4]:
# You can always call the help function which will provide information regarding the passed object
help(KeyfigureDataDownloader)

Help on class KeyfigureDataDownloader in module sap_ibp_connector.data_downloaders:

class KeyfigureDataDownloader(DataDownloader)
 |  KeyfigureDataDownloader(connector: sap_ibp_connector.connector.IBPConnector)
 |  
 |  Class for extracting key-figure data from SAP IBP
 |  
 |  Usage:
 |  
 |  with IBPConnector() as ibp_connector:
 |      downloader = KeyfigureDataDownloader(ibp_connector)
 |      result = downloader.download()
 |  
 |  Attributes
 |  ----------
 |  param connector:
 |      Instance of IBPConnector (wrapper around requests.Session)
 |  
 |  Method resolution order:
 |      KeyfigureDataDownloader
 |      DataDownloader
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, connector: sap_ibp_connector.connector.IBPConnector)
 |      Constructs all the necessary attributes for the KeyfigureDataDownloader object.
 |  
 |  download(self, query: Union[str, sap_ibp_connector.query.components.Query] = '', verbose=False, **kwargs) -> T

In [5]:
# Set up connection using IBPConnector
with IBPConnector(
    username=IBP_USERNAME_MD_KF,
    password=IBP_PASSWORD_MD_KF,
    planning_area=IBP_PLANNING_AREA,
    url=IBP_URL_KF,
) as connector:

    # Instantiate data downloader
    downloader = KeyfigureDataDownloader(connector)

    # Create a query
    select_query = select("PRDID", "PRDDESCR", "LOCREGION", "CUSTBUFFER2", "LOCID", "LOCDESCR", "PRDGROUP", "PRDGROUPID", "BRAND", "BRANDID", "DOSAGEFORM", "DOSAGEFORMID", "DOSAGE", "DOSAGEID", "PACKAGINGTYPE", "PACKAGINGTYPEID", "SUPPLYCHAINCAT", "LOCBUFFER3", "PERIODID3_TSTAMP", "CONSENSUSDEMANDQTYLM", "CONSENSUSDEMANDQTY", "ACTUALSQTY", "FINALSTATFCSTQTY", "CDPQ0MLAG", "CDPQ1MLAG", "CDPQ2MLAG", "CDPQ3MLAG", "CDPQ4MLAG", "CDPQ5MLAG", "FREEOFCHARGESALES", "SUMACTFOC", "STATISTICALFORECASTQTY3MLAG", "ZFINALSTATFCSTQTY")

    filter_query = AND(
        filter.eq("UOMTOID", "PC"),
        filter.ge("PERIODID3_TSTAMP", string_to_query_date("2020-09-01", "%Y-%m-%d")),
        filter.le("PERIODID3_TSTAMP", string_to_query_date("2022-08-01", "%Y-%m-%d")),
    )

    query = Query(select_query, filter_query)
    query = select_query + filter_query

    # Download data
    df, response = downloader.download_in_chunks(query=query)

InvalidSchema: No connection adapters were found for "Ellipsis/ZAPPLESI3?&$select=PRDID,PRDDESCR,LOCREGION,CUSTBUFFER2,LOCID,LOCDESCR,PRDGROUP,PRDGROUPID,BRAND,BRANDID,DOSAGEFORM,DOSAGEFORMID,DOSAGE,DOSAGEID,PACKAGINGTYPE,PACKAGINGTYPEID,SUPPLYCHAINCAT,LOCBUFFER3,PERIODID3_TSTAMP,CONSENSUSDEMANDQTYLM,CONSENSUSDEMANDQTY,ACTUALSQTY,FINALSTATFCSTQTY,CDPQ0MLAG,CDPQ1MLAG,CDPQ2MLAG,CDPQ3MLAG,CDPQ4MLAG,CDPQ5MLAG,FREEOFCHARGESALES,SUMACTFOC,STATISTICALFORECASTQTY3MLAG,ZFINALSTATFCSTQTY&$filter=((UOMTOID eq 'PC') and (PERIODID3_TSTAMP ge datetime'2020-09-01T00:00:00') and (PERIODID3_TSTAMP le datetime'2022-08-01T00:00:00'))&$skip=0&$top=100000"

In [ ]:
df.tail()

## Example of reading master data

In [ ]:
# CREDENTIALS PROD
IBP_USERNAME_MD_KF = ...
IBP_PASSWORD_MD_KF = ...
IBP_URL_KF = ...
IBP_PLANNING_AREA = "ZAPPLESI3"

In [ ]:
# Define the master data table you wish to read
masterdata_table = 'ZACLOCATIONPRODUCT'

In [ ]:
help(MasterdataDownloader)

In [ ]:
with IBPConnector(
    username=IBP_USERNAME_MD_KF,
    password=IBP_PASSWORD_MD_KF,
    planning_area=IBP_PLANNING_AREA,
    url=IBP_URL_KF,
) as ibp_connector:

    md_downloader = MasterdataDownloader(ibp_connector, masterdata_table=masterdata_table)
    df, response_results = md_downloader.download_in_chunks(chunk_size=1000)

## Example of writing key-figure data

In [ ]:
# Using DEV credentials for testing here

# CREDENTIALS DEV
IBP_USERNAME_MD_KF = ...
IBP_PASSWORD_MD_KF = ...
IBP_URL_KF = ...
IBP_PLANNING_AREA = "ZAPPLESI3"

In [ ]:
upload_df = pd.DataFrame(
    [
        ["114265", "AT50", "2022-09-01T00:00", "429.0"],
        ["114265", "AT50", "2022-10-01T00:00", "1129.0"],
    ],
    columns=["PRDID", "LOCID", "PERIODID3_TSTAMP", "ZAIAFCSTLOADPRODLOC"],
)

In [ ]:
upload_df.head()

In [ ]:
with IBPConnector(
    username=IBP_USERNAME_MD_KF,
    password=IBP_PASSWORD_MD_KF,
    planning_area=IBP_PLANNING_AREA,
    url=IBP_URL_KF,
) as ibp_connector:

    kf_uploader = KeyfigureDataUploader(ibp_connector)
    res = kf_uploader.upload(upload_df)

In [ ]:
res